In [96]:
import requests
import time
import re
import pandas as pd
import pprint
import pycrfsuite

import eli5
#import lime
#from lime.lime_text import LimeTextExplainer


from sklearn_crfsuite import CRF 
from sklearn.model_selection import cross_val_predict # train_test_split <- 쓰려면 쓰기
from sklearn_crfsuite .metrics import flat_classification_report # metrics: measure(Accuracy, score 등) 도구

from pycrfsuite_spacing import TemplateGenerator
from pycrfsuite_spacing import CharacterFeatureTransformer
from pycrfsuite_spacing import sent_to_xy
from bs4 import BeautifulSoup
from konlpy.tag import Kkma as kkm
from konlpy.tag import Okt
#from konlpy.tag import Mecab
#import Mecab
import tweepy


# score 정보 -> (원하는 것) 어떤 문장이 몇점인지를 예측 -> 이 값이 평균 이상이면 긍정, 평균 미만이면 부정
#            -> 긍/부정 으로만 학습하면 결과가 더 부정확할 수도. 
#            -> (현행) 평균 이상이면 100, 미만 0으로 분류해서 학습, 적용

# kkm 형태소 분석기에서 사전 조정해서, 칼퇴, 워라밸 같은 우리가 사용하는 단어로 수정하기.
# raw data : 맞춤법 교정.
# 조사 같은 연결어 제외하기. feature 추가(전전단어와 후후단어)
# CRF : N-gram ?????  kkm 사전을 형태소로 나눠놓고 하는 걸로. 

# 긍정/부정 리뷰 스크래핑+키워드 추출+사전으로 추가(feature) 
# 명사(를 비롯한 주요 단어)인 pos tag 만 뽑아서 feature에서 가중치 주기. 
# feature: pos tag에서 규칙 찾아서(eg. NNG+NNG) 여러 단어 연접하여 새롭게 긍/부정어 사전 돌리기 ??? <- 복잡. 
# 쪼개진 긍부정어 사전(만들어진 txt) 찾아보기.

# 조사 없애기? pos tag 포기. 띄어쓰기 교정 + 맞춤법 교정 + CRF ...? 
# feat -1, +1 < 최고로 나쁘다 -> 최고 로 나쁘 다 : 이런 식으로 CRF 특징이 무효화 됨.  

# CRF가 아니라, 다른 방식으로 해보는 게 어떤지? eg. review classification처럼 
# summarization + 빈도수 처리 + .... + N-gram 활용
# 시각화: eli5 

In [50]:
# 참고 사이트 https://yeo0.github.io/data/2018/09/24/5.-%EB%A1%9C%EA%B7%B8%EC%9D%B8%EC%9D%B4-%ED%95%84%EC%9A%94%ED%95%9C-%EC%82%AC%EC%9D%B4%ED%8A%B8%EC%97%90%EC%84%9C%EC%9D%98-%ED%81%AC%EB%A1%A4%EB%A7%81/
# 참고 사이트 2 https://hashcode.co.kr/questions/9084/%EC%9E%A1%ED%94%8C%EB%9E%98%EB%8B%9B-%EB%A1%9C%EA%B7%B8%EC%9D%B8-%ED%9B%84-%EC%8A%A4%ED%81%AC%EB%A0%88%EC%9D%B4%ED%95%91-%EB%AC%B8%EC%9D%98%EB%93%9C%EB%A6%BD%EB%8B%88%EB%8B%A4-%E3%85%9C%E3%85%9C

# 로그인 할 url
url = "https://www.jobplanet.co.kr/users/sign_in?_nav=gb"
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36'
headers = {'Content-type': 'application/json', 'Accept': 'text/plain', 'User-Agent':user_agent}
login_data = {'user':{'email':'hsmy31@hanyang.ac.kr', 'password':'rhkwpgksmswnd!', 'remember_me':'true'}}
session = requests.session()

# 로그인 실행
login_response = session.post(url, json = login_data, headers = headers)

reviews = []

In [51]:
def ind_reviews(code):
    url = "https://www.jobplanet.co.kr/reviews?&industry_id=" + str(code)
    response =  session.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    num = soup.find('span', class_='num') # 리뷰 개수 확인
    num = int(num.get_text().strip())
    import math, random
    pages = math.ceil(num / 10)
    page = random.sample(range(1, pages),30)
    reviews = {}
    for i in page: # 페이지를 랜덤으로 하여 300개의 리뷰 추출
        time.sleep(1)
        url = "https://www.jobplanet.co.kr/reviews?&industry_id=" + str(code) + "&page="+ str(i)
        response =  session.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        label = soup.find_all('h2', class_="us_label") # 한 줄 리뷰 추출
        # 추출한 리뷰에서 태그를 제외한 텍스트만 추출하여 labels 리스트에 저장
        labels = [label.get_text().strip() for label in label[:]]
        star = soup.find_all('div', class_="star_score") # 별점이 포함된 div 추출
        # 추출한 div에서 별점을 나타내는 width 속성의 숫자로 된 부분을 stars 리스트에 저장
        stars = re.findall('[0-9]+[.]+[0-9]', str(star))
        # reviews 딕셔너리에 '리뷰: 별점' 형식으로 추가
        for j in range(len(labels)):
            reviews[labels[j][5:-1]] = stars[j].replace('.0','')
    return reviews

In [52]:
dict_pos=[]
dict_neg=[]
for line in open("C:/MY/대학/3학년 2학기/텍스트마이닝/final project/word_dict/pos_pol_word.txt", 'rt', encoding='UTF8'):
    dict_pos.append(line.split('\n')[0])
    
for line in open("C:/MY/대학/3학년 2학기/텍스트마이닝/final project/word_dict/neg_pol_word.txt", 'rt', encoding='UTF8'):
    dict_neg.append(line.split('\n')[0])

In [53]:
# 각 산업군의 기업 리뷰 스크래핑
reviews1=dict(ind_reviews(1001))

In [54]:
reviews1

{'어지간하면 행정직으로 가라. 행정직이 최고야 얘들아...': '58',
 '전국의 문화재를 보존관리하는 업무를 할수있는곳, 청단위 중에서 기획, 정책업무가 과다하지 않음. 본청이 대전이고 전국 소속기관(외곽)으로 옮겨다닐수 있다는 단점이 있음. 지방근무를 해야함에도 불구하고 본격적이고 전문적으로 문화재업무를 할수있는 학예직에게 추천': '58',
 '연구직이지만 공공기업답게 짜디짠 월급을 주는 회사': '58',
 '그럭저럭 다닐만한 곳임. 급여수준은 보통이나 일이 편함': '58',
 '업무 난이도가 낮고 많은 시간을 확보할 수 있습니다.': '58',
 '보이기식 인재채용으로 처우개선이 필요함. 일자리 양성을 했다고는 하지만 실제론 주먹구구식 채용으로 이직률 높음': '58',
 '학교밖청소년 지원의 특수성 있으나 직원의 업무환경, 복지에 한계가 있음.': '58',
 '안정된 직장을 원한다면 추천드립니다. 워라밸을 챙길수 있다고 장담은 못하겠음…': '58',
 '수많은 민원과 왜 해야하는지 의문인 업무들이 있음. 지방조직이 많고 본부는 세종에': '58',
 '부서마다 다르겠지만 고요한 느낌?\r( 인턴이라 권한이 없어 이렇게 느낀걸수도)': '58',
 '직원 간 서로 존중해주는 분위기였습니다.': '68',
 '워라벨존중. 무난하게 일할수 있는 근무환경. 아르바이트지만 나쁘지않았음': '68',
 '남과 북이 공동으로 편찬하는 최초의 우리말 사전이라는 특수성': '68',
 '공공기관이므로 업무에 대한 스트레스나 강도가 강하지않고 한국과 농업은 떨어질수없는 관계이므로 미래성이 보장됨': '68',
 '자기 할일만 잘하면 눈치안보고 생활 할 수 있는 회사.': '68',
 '산림쪽으로는 가장 갈만한 공공기관이라고 생각함': '68',
 '대구가 연고지인 사람들은 평생 대구에서 살 수 있다는 게 장점인 직렬..': '68',
 '공기업이라 그런지 사기업 대비 비교적 여유있게 일했던 것 같음': '68',
 '엄청난 고용안정적인 직업. 워라밸 보장은 

In [55]:
text1=list(reviews1.keys())
score1=list(reviews1.values())

In [56]:
sum=0
for i in range(len(score1)):
    sum+=int(score1[i])
avr=sum/len(score1)
print("score 평균값: ", round(avr,3))

score 평균값:  60.527


In [57]:
import site; site.getsitepackages()


['c:\\users\\sj\\appdata\\local\\programs\\python\\python39',
 'c:\\users\\sj\\appdata\\local\\programs\\python\\python39\\lib\\site-packages']

In [58]:
sent_num=0
text1_sent=[]
kkm=kkm()
#mecab = Mecab(dicpath=r"C:/mecab/mecab-ko-dic")
for i in range(len(text1)):
    text1_pos=kkm.pos(text1[i])
    sent_num+=1
    for t in text1_pos:
        n=list(t)
        
        sent_number="Sentence: {}".format(sent_num)
        n.insert(0, sent_number)
        '''
        if n[1] in dict_pos: 
            pos_dict=True
        else: 
            pos_dict=False
        n.insert(3, pos_dict)
        if n[1] in dict_neg: 
            neg_dict=True
        else: 
            neg_dict=False
        n.insert(4, neg_dict)
        '''
        if float(score1[(sent_num-1)]) > avr:
            n.insert(3, '100')
        else: 
            n.insert(3, '0')
        
        #n.insert(5, score1[(sent_num-1)])
        
        text1_sent.append(n)
        

pprint.pprint(text1_sent)

[['Sentence: 1', '어지', 'NNG', '0'],
 ['Sentence: 1', '간하', 'VV', '0'],
 ['Sentence: 1', '면', 'ECE', '0'],
 ['Sentence: 1', '행정직', 'NNG', '0'],
 ['Sentence: 1', '으로', 'JKM', '0'],
 ['Sentence: 1', '가', 'VV', '0'],
 ['Sentence: 1', '라', 'ECD', '0'],
 ['Sentence: 1', '.', 'SF', '0'],
 ['Sentence: 1', '행정직', 'NNG', '0'],
 ['Sentence: 1', '이', 'JKS', '0'],
 ['Sentence: 1', '최고', 'NNG', '0'],
 ['Sentence: 1', '야', 'JX', '0'],
 ['Sentence: 1', '얘들', 'UN', '0'],
 ['Sentence: 1', '아', 'JKI', '0'],
 ['Sentence: 1', '...', 'SE', '0'],
 ['Sentence: 2', '전국', 'NNG', '0'],
 ['Sentence: 2', '의', 'JKG', '0'],
 ['Sentence: 2', '문화재', 'NNG', '0'],
 ['Sentence: 2', '를', 'JKO', '0'],
 ['Sentence: 2', '보존', 'NNG', '0'],
 ['Sentence: 2', '관리', 'NNG', '0'],
 ['Sentence: 2', '하', 'XSV', '0'],
 ['Sentence: 2', '는', 'ETD', '0'],
 ['Sentence: 2', '업무', 'NNG', '0'],
 ['Sentence: 2', '를', 'JKO', '0'],
 ['Sentence: 2', '하', 'VV', '0'],
 ['Sentence: 2', 'ㄹ', 'ETD', '0'],
 ['Sentence: 2', '수', 'NNB', '0'],
 ['Sentenc

 ['Sentence: 80', '크', 'VA', '100'],
 ['Sentence: 80', '게', 'ECD', '100'],
 ['Sentence: 80', '나쁘', 'VA', '100'],
 ['Sentence: 80', '지', 'ECD', '100'],
 ['Sentence: 80', '도', 'JX', '100'],
 ['Sentence: 80', '않', 'VXV', '100'],
 ['Sentence: 80', '다', 'EFN', '100'],
 ['Sentence: 81', '검사', 'NNG', '0'],
 ['Sentence: 81', '의', 'JKG', '0'],
 ['Sentence: 81', ',', 'SP', '0'],
 ['Sentence: 81', '검사', 'NNG', '0'],
 ['Sentence: 81', '에', 'JKM', '0'],
 ['Sentence: 81', '의하', 'VV', '0'],
 ['Sentence: 81', 'ㄴ', 'ETD', '0'],
 ['Sentence: 81', ',', 'SP', '0'],
 ['Sentence: 81', '검사', 'NNG', '0'],
 ['Sentence: 81', '를', 'JKO', '0'],
 ['Sentence: 81', '위하', 'VV', '0'],
 ['Sentence: 81', 'ㄴ', 'ETD', '0'],
 ['Sentence: 81', '조직', 'NNG', '0'],
 ['Sentence: 82', '공기업', 'NNG', '0'],
 ['Sentence: 82', '의', 'JKG', '0'],
 ['Sentence: 82', '장단점', 'NNG', '0'],
 ['Sentence: 82', '을', 'JKO', '0'],
 ['Sentence: 82', '그대로', 'MAG', '0'],
 ['Sentence: 82', '답습', 'NNG', '0'],
 ['Sentence: 82', '하', 'XSV', '0'],
 ['Sent

 ['Sentence: 133', '고', 'ECE', '100'],
 ['Sentence: 133', '유한', 'NNG', '100'],
 ['Sentence: 133', '분위기', 'NNG', '100'],
 ['Sentence: 133', '의', 'JKG', '100'],
 ['Sentence: 133', '공', 'NNG', '100'],
 ['Sentence: 133', '조직', 'NNG', '100'],
 ['Sentence: 133', '..', 'SW', '100'],
 ['Sentence: 133', '그렇', 'VA', '100'],
 ['Sentence: 133', '기에', 'ECD', '100'],
 ['Sentence: 133', '공무원', 'NNG', '100'],
 ['Sentence: 133', '이', 'JKS', '100'],
 ['Sentence: 133', '아니', 'VV', '100'],
 ['Sentence: 133', 'ㄴ', 'ETD', '100'],
 ['Sentence: 133', '자', 'NNB', '100'],
 ['Sentence: 133', '는', 'JX', '100'],
 ['Sentence: 133', '소외감', 'NNG', '100'],
 ['Sentence: 133', '을', 'JKO', '100'],
 ['Sentence: 133', '느낌', 'NNG', '100'],
 ['Sentence: 134', '광주', 'NNG', '100'],
 ['Sentence: 134', '헤', 'VV', '100'],
 ['Sentence: 134', 'ㄹ', 'ETD', '100'],
 ['Sentence: 134', '게이트', 'NNG', '100'],
 ['Sentence: 134', '구청', 'NNG', '100'],
 ['Sentence: 134', '투', 'NNG', '100'],
 ['Sentence: 134', '톱', 'NNG', '100'],
 ['Sentence: 

 ['Sentence: 176', '을', 'JKO', '100'],
 ['Sentence: 176', '진행', 'NNG', '100'],
 ['Sentence: 176', '하', 'XSV', '100'],
 ['Sentence: 176', '기', 'ETN', '100'],
 ['Sentence: 176', '보다', 'JKM', '100'],
 ['Sentence: 176', '본인', 'NNG', '100'],
 ['Sentence: 176', '의', 'JKG', '100'],
 ['Sentence: 176', '만족', 'NNG', '100'],
 ['Sentence: 176', '과', 'JKM', '100'],
 ['Sentence: 176', '쓸데없', 'VA', '100'],
 ['Sentence: 176', '는', 'ETD', '100'],
 ['Sentence: 176', '경쟁심', 'NNG', '100'],
 ['Sentence: 176', '으로', 'JKM', '100'],
 ['Sentence: 176', '인하', 'VV', '100'],
 ['Sentence: 176', '어', 'ECS', '100'],
 ['Sentence: 176', '본인', 'NNG', '100'],
 ['Sentence: 176', '캐퍼', 'UN', '100'],
 ['Sentence: 176', '에', 'JKM', '100'],
 ['Sentence: 176', '맞', 'VV', '100'],
 ['Sentence: 176', '지', 'ECD', '100'],
 ['Sentence: 176', '않', 'VXV', '100'],
 ['Sentence: 176', '는', 'ETD', '100'],
 ['Sentence: 176', '사업', 'NNG', '100'],
 ['Sentence: 176', '들', 'XSN', '100'],
 ['Sentence: 176', '을', 'JKO', '100'],
 ['Sentence: 176

 ['Sentence: 240', '...', 'SE', '0'],
 ['Sentence: 241', '공무', 'NNG', '100'],
 ['Sentence: 241', '직', 'NNG', '100'],
 ['Sentence: 241', '으로', 'JKM', '100'],
 ['Sentence: 241', '가', 'VV', '100'],
 ['Sentence: 241', '었', 'EPT', '100'],
 ['Sentence: 241', '으나', 'ECE', '100'],
 ['Sentence: 241', '여러모로', 'MAG', '100'],
 ['Sentence: 241', '힘들', 'VA', '100'],
 ['Sentence: 241', '고', 'ECE', '100'],
 ['Sentence: 241', '안', 'MAG', '100'],
 ['Sentence: 241', '맞', 'VV', '100'],
 ['Sentence: 241', '아서', 'ECD', '100'],
 ['Sentence: 241', '때리', 'VV', '100'],
 ['Sentence: 241', '어', 'ECS', '100'],
 ['Sentence: 241', '침요', 'NNG', '100'],
 ['Sentence: 242', '준', 'XPN', '100'],
 ['Sentence: 242', '공공', 'NNG', '100'],
 ['Sentence: 242', '기관', 'NNG', '100'],
 ['Sentence: 242', '으로', 'JKM', '100'],
 ['Sentence: 242', '비교', 'NNG', '100'],
 ['Sentence: 242', '적', 'XSN', '100'],
 ['Sentence: 242', '안정적', 'NNG', '100'],
 ['Sentence: 242', '이', 'VCP', '100'],
 ['Sentence: 242', 'ㄴ', 'ETD', '100'],
 ['Sentence: 2

 ['Sentence: 292', 'ㄴ', 'ETD', '100'],
 ['Sentence: 292', '프로그램', 'NNG', '100'],
 ['Sentence: 292', '을', 'JKO', '100'],
 ['Sentence: 292', '운영', 'NNG', '100'],
 ['Sentence: 292', '중', 'NNB', '100'],
 ['Sentence: 292', '이', 'VCP', '100'],
 ['Sentence: 292', '라', 'ECD', '100'],
 ['Sentence: 292', '좋', 'VA', '100'],
 ['Sentence: 292', '다', 'EFN', '100'],
 ['Sentence: 292', '.', 'SF', '100'],
 ['Sentence: 293', '대한민국', 'NNG', '100'],
 ['Sentence: 293', '식량', 'NNG', '100'],
 ['Sentence: 293', '산업', 'NNG', '100'],
 ['Sentence: 293', '의', 'JKG', '100'],
 ['Sentence: 293', '최전선', 'NNG', '100'],
 ['Sentence: 293', '에서', 'JKM', '100'],
 ['Sentence: 293', '분투', 'NNG', '100'],
 ['Sentence: 293', '하', 'XSV', '100'],
 ['Sentence: 293', '는', 'ETD', '100'],
 ['Sentence: 293', '해군', 'NNG', '100'],
 ['Sentence: 293', 'UDT', 'OL', '100'],
 ['Sentence: 293', '같', 'VA', '100'],
 ['Sentence: 293', '은', 'ETD', '100'],
 ['Sentence: 293', '소수', 'NNG', '100'],
 ['Sentence: 293', '정예', 'NNG', '100'],
 ['Sentence

In [59]:
text1_sent

[['Sentence: 1', '어지', 'NNG', '0'],
 ['Sentence: 1', '간하', 'VV', '0'],
 ['Sentence: 1', '면', 'ECE', '0'],
 ['Sentence: 1', '행정직', 'NNG', '0'],
 ['Sentence: 1', '으로', 'JKM', '0'],
 ['Sentence: 1', '가', 'VV', '0'],
 ['Sentence: 1', '라', 'ECD', '0'],
 ['Sentence: 1', '.', 'SF', '0'],
 ['Sentence: 1', '행정직', 'NNG', '0'],
 ['Sentence: 1', '이', 'JKS', '0'],
 ['Sentence: 1', '최고', 'NNG', '0'],
 ['Sentence: 1', '야', 'JX', '0'],
 ['Sentence: 1', '얘들', 'UN', '0'],
 ['Sentence: 1', '아', 'JKI', '0'],
 ['Sentence: 1', '...', 'SE', '0'],
 ['Sentence: 2', '전국', 'NNG', '0'],
 ['Sentence: 2', '의', 'JKG', '0'],
 ['Sentence: 2', '문화재', 'NNG', '0'],
 ['Sentence: 2', '를', 'JKO', '0'],
 ['Sentence: 2', '보존', 'NNG', '0'],
 ['Sentence: 2', '관리', 'NNG', '0'],
 ['Sentence: 2', '하', 'XSV', '0'],
 ['Sentence: 2', '는', 'ETD', '0'],
 ['Sentence: 2', '업무', 'NNG', '0'],
 ['Sentence: 2', '를', 'JKO', '0'],
 ['Sentence: 2', '하', 'VV', '0'],
 ['Sentence: 2', 'ㄹ', 'ETD', '0'],
 ['Sentence: 2', '수', 'NNB', '0'],
 ['Sentenc

In [115]:
text2_del_SW = []
for i in range(len(text2_sent)):
    for t in text1_sent : 
        if ( (t[2] != 'ETD') | (t[2] !='SF') | (t[2] !='SP') | (t[2] !='XSV') 
            | (t[2][:2] !='JK')   ):
            text2_del_SW.append(t)

In [116]:
text2_del_SW

[['Sentence: 1', '어지', 'NNG', '0'],
 ['Sentence: 1', '간하', 'VV', '0'],
 ['Sentence: 1', '면', 'ECE', '0'],
 ['Sentence: 1', '행정직', 'NNG', '0'],
 ['Sentence: 1', '으로', 'JKM', '0'],
 ['Sentence: 1', '가', 'VV', '0'],
 ['Sentence: 1', '라', 'ECD', '0'],
 ['Sentence: 1', '.', 'SF', '0'],
 ['Sentence: 1', '행정직', 'NNG', '0'],
 ['Sentence: 1', '이', 'JKS', '0'],
 ['Sentence: 1', '최고', 'NNG', '0'],
 ['Sentence: 1', '야', 'JX', '0'],
 ['Sentence: 1', '얘들', 'UN', '0'],
 ['Sentence: 1', '아', 'JKI', '0'],
 ['Sentence: 1', '...', 'SE', '0'],
 ['Sentence: 2', '전국', 'NNG', '0'],
 ['Sentence: 2', '의', 'JKG', '0'],
 ['Sentence: 2', '문화재', 'NNG', '0'],
 ['Sentence: 2', '를', 'JKO', '0'],
 ['Sentence: 2', '보존', 'NNG', '0'],
 ['Sentence: 2', '관리', 'NNG', '0'],
 ['Sentence: 2', '하', 'XSV', '0'],
 ['Sentence: 2', '는', 'ETD', '0'],
 ['Sentence: 2', '업무', 'NNG', '0'],
 ['Sentence: 2', '를', 'JKO', '0'],
 ['Sentence: 2', '하', 'VV', '0'],
 ['Sentence: 2', 'ㄹ', 'ETD', '0'],
 ['Sentence: 2', '수', 'NNB', '0'],
 ['Sentenc

In [ ]:
text2_del_SW = []
for i in range(len(text2_sent)):
    for t in text2_sent : 
        if t[1] != 'ETD':
            text2_del_SW.append()

In [60]:
dict_pos

['출처',
 '1) 표준국어대사전 뜻풀이',
 '-> https://github.com/mrchypark/stdkor',
 '2) 신조어',
 '-> https://ko.wikipedia.org/wiki/%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD%EC%9D%98_%EC%9D%B8%ED%84%B0%EB%84%B7_%EC%8B%A0%EC%A1%B0%EC%96%B4_%EB%AA%A9%EB%A1%9D',
 '3) 이모티콘',
 '-> https://ko.wikipedia.org/wiki/%EC%9D%B4%EB%AA%A8%ED%8B%B0%EC%BD%98',
 '4) SentiWordNet_3.0.0_20130122',
 '-> http://sentiwordnet.isti.cnr.it/',
 '5) SenticNet-5.0',
 '-> http://sentic.net/',
 '6) 감정단어사전0603',
 '-> http://datascience.khu.ac.kr/board/bbs/board.php?bo_table=05_01&wr_id=91 ',
 '7) 김은영, “국어 감정동사 연구”, 2004.02, 학위논문(박사) - 전남대학교 국어국문학과 대학원',
 '',
 '',
 '단어 수 : 4863',
 '극성 : 긍정',
 '',
 '(-;',
 '(^^)',
 '(^-^)',
 '(^^*',
 '(^_^)',
 '(^o^)',
 '*^^*',
 '/^o^\\',
 ':(',
 ":'-(",
 ':-(',
 ':-)',
 ':|',
 ';',
 ';)',
 ';-)',
 '^///^',
 '^^',
 '^^//',
 '<3',
 '=)',
 'XD',
 'ㄱㅅ',
 'ㄱㅇㄷ',
 '가격이 싸다',
 '가까이 사귀어',
 '가까이하다',
 '가다듬어',
 '가다듬어 수습하는',
 '가다듬어 수습하다',
 '가다듬어 정하다',
 '가당히',
 '가뜬하게',
 '가라앉혀 바로잡다',
 '가르치며',
 '가르침',
 '가벼운 마음으로',
 '가볍게 웃

In [61]:
dict_neg

['출처',
 '1) 표준국어대사전 뜻풀이',
 '-> https://github.com/mrchypark/stdkor',
 '2) 신조어',
 '-> https://ko.wikipedia.org/wiki/%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD%EC%9D%98_%EC%9D%B8%ED%84%B0%EB%84%B7_%EC%8B%A0%EC%A1%B0%EC%96%B4_%EB%AA%A9%EB%A1%9D',
 '3) 이모티콘',
 '-> https://ko.wikipedia.org/wiki/%EC%9D%B4%EB%AA%A8%ED%8B%B0%EC%BD%98',
 '4) SentiWordNet_3.0.0_20130122',
 '-> http://sentiwordnet.isti.cnr.it/',
 '5) SenticNet-5.0',
 '-> http://sentic.net/',
 '6) 감정단어사전0603',
 '-> http://datascience.khu.ac.kr/board/bbs/board.php?bo_table=05_01&wr_id=91 ',
 '7) 김은영, “국어 감정동사 연구”, 2004.02, 학위논문(박사) - 전남대학교 국어국문학과 대학원',
 '',
 '',
 '단어 수 : 9826',
 '극성 : 부정',
 '',
 '가난',
 '가난뱅이',
 '가난살이',
 '가난살이하다',
 '가난설음',
 '가난에',
 '가난에 쪼들려서',
 '가난하게',
 '가난하고',
 '가난하고 어렵다',
 '가난하고 천한',
 '가난하다',
 '가난하여',
 '가난한',
 '가난한 데다가',
 '가난한 사람을',
 '가난히',
 '가년스럽다',
 '가녕스럽다',
 '가누지 못하고',
 '가누지 못하다',
 '가누지 못할',
 '가누지 못할 정도로',
 '가누지 못함',
 '가눌 수 없게',
 '가눌 수 없을',
 '가눌 수 없이',
 '가당찮다',
 '가당찮이',
 '가두거나 해치거나',
 '가망이 없게',
 '가슴 아파하다',
 '가슴 아파함',

In [62]:
data = pd.DataFrame(text1_sent) # data를 pandas로 변환

In [63]:
data.columns = ['Sentence #', 'Word', 'POS', 'score']

In [64]:
data.head(10)

,Sentence #,Word,POS,score
0,Sentence: 1,어지,NNG,0
1,Sentence: 1,간하,VV,0
2,Sentence: 1,면,ECE,0
3,Sentence: 1,행정직,NNG,0
4,Sentence: 1,으로,JKM,0
5,Sentence: 1,가,VV,0
6,Sentence: 1,라,ECD,0
7,Sentence: 1,.,SF,0
8,Sentence: 1,행정직,NNG,0
9,Sentence: 1,이,JKS,0


In [65]:
data.tail(10)

,Sentence #,Word,POS,score
7321,Sentence: 300,원인,NNG,100
7322,Sentence: 300,상대,NNG,100
7323,Sentence: 300,는,JX,100
7324,Sentence: 300,어,VV,100
7325,Sentence: 300,어,ECS,100
7326,Sentence: 300,디,VV,100
7327,Sentence: 300,ㄹ,ETD,100
7328,Sentence: 300,가나,NNG,100
7329,Sentence: 300,있음,NNG,100
7330,Sentence: 300,.,SF,100


In [66]:
data.isnull().sum() # 비어있는 값 확인 결과: 없음

Sentence #    0
Word          0
POS           0
score         0
dtype: int64

In [67]:
words = list(set(data["Word"].values))
n_words = len(words) # 고유한 단어들이 1784개(문장부호 포함)
n_words

1479

In [68]:
# 데이터에서 문장을 원하는 형태로 포매팅하여 추출하는 클래스
# Word, POS -> data를 여러 함수로 씌워서 봄. 
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        # for문 사용법 보기. lambda: 일종의 함수(적용하는), data에 다음과 같은 기능 적용
        agg_func = lambda s: [(w, p, s) for w, p, s in zip(s["Word"].values.tolist(),
                                                                     s["POS"].values.tolist(),
                                                                     s["score"].values.tolist())]
        # sentence 단위로 grouped
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        # 그룹화된 데이터를 다시 sentences로 저장
        self.sentences = [s for s in self.grouped]
        print(self.grouped)
        self.sentences
    
    # get_next: 다음 문장 처리
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None


In [69]:
# 전체 문장 추출
getter = SentenceGetter(data)

Sentence #
Sentence: 1      [(어지, NNG, 0), (간하, VV, 0), (면, ECE, 0), (행정직,...
Sentence: 10     [(부서, NNG, 0), (마다, JX, 0), (다르, VA, 0), (겠, E...
Sentence: 100    [(지역, NNG, 0), (의, JKG, 0), (고인물, NNG, 0), (대잔...
Sentence: 101    [(여직원, NNG, 100), (의, JKG, 100), (비율, NNG, 100...
Sentence: 102    [(국내, NNG, 100), (보건, NNG, 100), (정책, NNG, 100...
                                       ...                        
Sentence: 95     [(시험, NNG, 100), (연구원, NNG, 100), (이, VCP, 100...
Sentence: 96     [(복지, NNG, 100), (및, MAG, 100), (급여, NNG, 100)...
Sentence: 97     [(워라밸, UN, 100), (이, JKS, 100), (충족, NNG, 100)...
Sentence: 98     [(일하, VV, 0), (는, ETD, 0), (사람, NNG, 0), (,, S...
Sentence: 99     [(대한민국, NNG, 0), (창작, NNG, 0), (공연, NNG, 0), (...
Length: 300, dtype: object


In [70]:
# 문장 하나
sent = getter.get_next()

In [71]:
# 추출한 문장 확인(적용 결과)
print(sent)

[('어지', 'NNG', '0'), ('간하', 'VV', '0'), ('면', 'ECE', '0'), ('행정직', 'NNG', '0'), ('으로', 'JKM', '0'), ('가', 'VV', '0'), ('라', 'ECD', '0'), ('.', 'SF', '0'), ('행정직', 'NNG', '0'), ('이', 'JKS', '0'), ('최고', 'NNG', '0'), ('야', 'JX', '0'), ('얘들', 'UN', '0'), ('아', 'JKI', '0'), ('...', 'SE', '0')]


In [72]:
# 전체 문장
sentences = getter.sentences

In [73]:
print(len(sentences)) # 개수 확인

# 문장 하나 확인
print(sentences[58])
print(sentences[:2]) # list의 list 형태로 여러 개 표시. 


# 각각 word, pos, tag, pare로 만들어서 결과 확인

300
[('최악', 'NNG', '0'), ('의', 'JKG', '0'), ('회사', 'NNG', '0'), ('.', 'SF', '0'), ('직원', 'NNG', '0'), ('들', 'XSN', '0'), ('을', 'JKO', '0'), ('위하', 'VV', '0'), ('ㄴ', 'ETD', '0'), ('복지', 'NNG', '0'), ('는', 'JX', '0'), ('있', 'VV', '0'), ('으나', 'ECE', '0'), ('그것', 'NP', '0'), ('을', 'JKO', '0'), ('누리', 'VV', '0'), ('ㄹ', 'ETD', '0'), ('새', 'NNG', '0'), ('가', 'JKS', '0'), ('없', 'VA', '0'), ('음', 'ETN', '0'), ('.', 'SF', '0'), ('왜', 'MAG', '0'), ('?', 'SF', '0'), ('야근', 'NNG', '0'), ('하', 'XSV', '0'), ('느라', 'ECD', '0'), ('..', 'SW', '0')]
[[('어지', 'NNG', '0'), ('간하', 'VV', '0'), ('면', 'ECE', '0'), ('행정직', 'NNG', '0'), ('으로', 'JKM', '0'), ('가', 'VV', '0'), ('라', 'ECD', '0'), ('.', 'SF', '0'), ('행정직', 'NNG', '0'), ('이', 'JKS', '0'), ('최고', 'NNG', '0'), ('야', 'JX', '0'), ('얘들', 'UN', '0'), ('아', 'JKI', '0'), ('...', 'SE', '0')], [('부서', 'NNG', '0'), ('마다', 'JX', '0'), ('다르', 'VA', '0'), ('겠', 'EPT', '0'), ('지만', 'ECE', '0'), ('고요', 'NNG', '0'), ('하', 'XSV', '0'), ('ㄴ', 'ETD', '0'), ('느낌', 'NNG',

In [95]:
# 단어를 features화
# "feat의 이름" : "value"
# 하나의 단어에서 다양한 feature(단어, POS tag, lables 등) 확인을 위해 문장 전체를 넣어줌. 

def word2features(sent, i):
    word = sent[i][0]   # index0의 단어를 가져옴
    postag = sent[i][1] # index1의 단어를 가져옴
    score = sent[i][2]

# postag 유지
# postag2 = kkma.pos(word)
# 
    features = {
        'bias': 1.0,
        'postag': postag,
        'postag[:2]': postag[:2],
        'dict_pos' : True if word in dict_pos else False,
        'dict_neg' : True if word in dict_neg else False,
        'word.len' : len(word)
        
# 특징!!!! 
# 
#'dict_time': word.lower() in dict_time # -> true or false로
# character 개수 
# 'word.len' : len(word)
    }
    
    # 직전의 단어에 대해서도 똑같이 적용
    if i > 0:
        word1 = sent[i-1][0] # i-1: 직전의 단어
        postag1 = sent[i-1][1]
        # update: 기존에서 추가! (하여 새로운 dictionary를 만드는 결과)
        features.update({
            '-1:postag': postag,
            'dict_pos' : True if word in dict_pos else False,
            'dict_neg' : True if word in dict_neg else False,
            '-1:word.len' : len(word)
        })
    #else:
    #    features['BOS'] = True # BOS: Beginning Of Sentence

    # 다음 단어에 대해서도 똑같이 처리
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:postag': postag,
            'dict_pos' : True if word in dict_pos else False,
            'dict_neg' : True if word in dict_neg else False,
            '+1:word.len' : len(word)
        })
    
    #else:
    #    features['EOS'] = True  # EOS: end of sentence
        
    # bi-그램에 대해서도 똑같이 처리
    
    return features

# sentence 전체에 적용하기 위한 함수
def sent2features(sent):
    for i in range(len(sent)):
        print(word2features(sent, i))
    return [word2features(sent, i) for i in range(len(sent))] # 0~i(index의 수)까지 적용 -> sentence에 적용
# == features = [word2features(sent, i) for i in range(len(sent))]
# return features

def sent2score(sent): # sent를 읽어서 for문을 돌림. label만 갖고 옴
    return [score for token, postag, score in sent]
# 지금은 3개가 tuple로 있음.

def sent2tokens(sent):
    return [token for token, postag, score in sent]

In [75]:
sent2score(sent)

['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']

In [76]:
print(sent2features(sent))

{'bias': 1.0, 'postag': 'NNG', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '+1:postag': 'NNG', '+1:word.len': 2}
{'bias': 1.0, 'postag': 'VV', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '-1:postag': 'VV', '-1:word.len': 2, '+1:postag': 'VV', '+1:word.len': 2}
{'bias': 1.0, 'postag': 'ECE', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'ECE', '-1:word.len': 1, '+1:postag': 'ECE', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'NNG', 'dict_pos': False, 'dict_neg': False, 'word.len': 3, '-1:postag': 'NNG', '-1:word.len': 3, '+1:postag': 'NNG', '+1:word.len': 3}
{'bias': 1.0, 'postag': 'JKM', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '-1:postag': 'JKM', '-1:word.len': 2, '+1:postag': 'JKM', '+1:word.len': 2}
{'bias': 1.0, 'postag': 'VV', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'VV', '-1:word.len': 1, '+1:postag': 'VV', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'ECD', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, 

In [77]:
X = [sent2features(s) for s in sentences] # sentence마다 feature로 바뀌고
y = [sent2score(s) for s in sentences] # sentence마다 score로 바뀜

{'bias': 1.0, 'postag': 'NNG', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '+1:postag': 'NNG', '+1:word.len': 2}
{'bias': 1.0, 'postag': 'VV', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '-1:postag': 'VV', '-1:word.len': 2, '+1:postag': 'VV', '+1:word.len': 2}
{'bias': 1.0, 'postag': 'ECE', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'ECE', '-1:word.len': 1, '+1:postag': 'ECE', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'NNG', 'dict_pos': False, 'dict_neg': False, 'word.len': 3, '-1:postag': 'NNG', '-1:word.len': 3, '+1:postag': 'NNG', '+1:word.len': 3}
{'bias': 1.0, 'postag': 'JKM', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '-1:postag': 'JKM', '-1:word.len': 2, '+1:postag': 'JKM', '+1:word.len': 2}
{'bias': 1.0, 'postag': 'VV', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'VV', '-1:word.len': 1, '+1:postag': 'VV', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'ECD', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, 

{'bias': 1.0, 'postag': 'EFN', 'dict_pos': False, 'dict_neg': False, 'word.len': 3, '-1:postag': 'EFN', '-1:word.len': 3, '+1:postag': 'EFN', '+1:word.len': 3}
{'bias': 1.0, 'postag': 'SF', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'SF', '-1:word.len': 1}
{'bias': 1.0, 'postag': 'NNG', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '+1:postag': 'NNG', '+1:word.len': 2}
{'bias': 1.0, 'postag': 'NNG', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '-1:postag': 'NNG', '-1:word.len': 2, '+1:postag': 'NNG', '+1:word.len': 2}
{'bias': 1.0, 'postag': 'NNG', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '-1:postag': 'NNG', '-1:word.len': 2, '+1:postag': 'NNG', '+1:word.len': 2}
{'bias': 1.0, 'postag': 'JKM', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'JKM', '-1:word.len': 1, '+1:postag': 'JKM', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'NNG', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '-1:postag': 'NNG', '-1:word.len':

{'bias': 1.0, 'postag': 'JKM', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'JKM', '-1:word.len': 1, '+1:postag': 'JKM', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'VV', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '-1:postag': 'VV', '-1:word.len': 2, '+1:postag': 'VV', '+1:word.len': 2}
{'bias': 1.0, 'postag': 'ECS', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'ECS', '-1:word.len': 1, '+1:postag': 'ECS', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'UN', 'dict_pos': False, 'dict_neg': False, 'word.len': 3, '-1:postag': 'UN', '-1:word.len': 3, '+1:postag': 'UN', '+1:word.len': 3}
{'bias': 1.0, 'postag': 'JKS', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'JKS', '-1:word.len': 1, '+1:postag': 'JKS', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'MDT', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '-1:postag': 'MDT', '-1:word.len': 2, '+1:postag': 'MDT', '+1:word.len': 2}
{'bias': 1.0, 'postag': 'NNG', 'dict_pos': Fal

{'bias': 1.0, 'postag': 'VA', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '+1:postag': 'VA', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'ETD', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'ETD', '-1:word.len': 1, '+1:postag': 'ETD', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'NNG', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '-1:postag': 'NNG', '-1:word.len': 2, '+1:postag': 'NNG', '+1:word.len': 2}
{'bias': 1.0, 'postag': 'JC', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'JC', '-1:word.len': 1, '+1:postag': 'JC', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'NNG', 'dict_pos': False, 'dict_neg': False, 'word.len': 3, '-1:postag': 'NNG', '-1:word.len': 3, '+1:postag': 'NNG', '+1:word.len': 3}
{'bias': 1.0, 'postag': 'NNB', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'NNB', '-1:word.len': 1, '+1:postag': 'NNB', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'JKG', 'dict_pos': False, 'dict_neg': False, 'word.len': 1,

{'bias': 1.0, 'postag': 'VXV', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'VXV', '-1:word.len': 1, '+1:postag': 'VXV', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'EFN', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'EFN', '-1:word.len': 1, '+1:postag': 'EFN', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'JX', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'JX', '-1:word.len': 1, '+1:postag': 'JX', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'VXV', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'VXV', '-1:word.len': 1, '+1:postag': 'VXV', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'ECE', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '-1:postag': 'ECE', '-1:word.len': 2, '+1:postag': 'ECE', '+1:word.len': 2}
{'bias': 1.0, 'postag': 'SP', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'SP', '-1:word.len': 1, '+1:postag': 'SP', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'NNB', 'dict_pos': Fal

...

{'bias': 1.0, 'postag': 'NNG', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '-1:postag': 'NNG', '-1:word.len': 2, '+1:postag': 'NNG', '+1:word.len': 2}
{'bias': 1.0, 'postag': 'XSV', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'XSV', '-1:word.len': 1, '+1:postag': 'XSV', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'ECD', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '-1:postag': 'ECD', '-1:word.len': 2, '+1:postag': 'ECD', '+1:word.len': 2}
{'bias': 1.0, 'postag': 'VV', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'VV', '-1:word.len': 1, '+1:postag': 'VV', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'ETD', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'ETD', '-1:word.len': 1, '+1:postag': 'ETD', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'NNG', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'NNG', '-1:word.len': 1, '+1:postag': 'NNG', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'JKS', 'dict_pos': 

{'bias': 1.0, 'postag': 'VA', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '+1:postag': 'VA', '+1:word.len': 2}
{'bias': 1.0, 'postag': 'ETD', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'ETD', '-1:word.len': 1, '+1:postag': 'ETD', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'NNG', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '-1:postag': 'NNG', '-1:word.len': 2, '+1:postag': 'NNG', '+1:word.len': 2}
{'bias': 1.0, 'postag': 'JKM', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'JKM', '-1:word.len': 1, '+1:postag': 'JKM', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'MAG', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'MAG', '-1:word.len': 1, '+1:postag': 'MAG', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'VV', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'VV', '-1:word.len': 1, '+1:postag': 'VV', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'ECD', 'dict_pos': False, 'dict_neg': False, 'word.len': 2,

{'bias': 1.0, 'postag': 'NNM', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'NNM', '-1:word.len': 1, '+1:postag': 'NNM', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'VV', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'VV', '-1:word.len': 1, '+1:postag': 'VV', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'ETN', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'ETN', '-1:word.len': 1}
{'bias': 1.0, 'postag': 'NNG', 'dict_pos': False, 'dict_neg': False, 'word.len': 3, '+1:postag': 'NNG', '+1:word.len': 3}
{'bias': 1.0, 'postag': 'NNG', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '-1:postag': 'NNG', '-1:word.len': 2, '+1:postag': 'NNG', '+1:word.len': 2}
{'bias': 1.0, 'postag': 'NNB', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'NNB', '-1:word.len': 1, '+1:postag': 'NNB', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'NNG', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '-1:postag': 'NNG', '-1:word.len': 

{'bias': 1.0, 'postag': 'NNG', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '+1:postag': 'NNG', '+1:word.len': 2}
{'bias': 1.0, 'postag': 'MAG', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'MAG', '-1:word.len': 1, '+1:postag': 'MAG', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'NNG', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '-1:postag': 'NNG', '-1:word.len': 2, '+1:postag': 'NNG', '+1:word.len': 2}
{'bias': 1.0, 'postag': 'JKS', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'JKS', '-1:word.len': 1, '+1:postag': 'JKS', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'NNG', 'dict_pos': True, 'dict_neg': False, 'word.len': 2, '-1:postag': 'NNG', '-1:word.len': 2, '+1:postag': 'NNG', '+1:word.len': 2}
{'bias': 1.0, 'postag': 'XSA', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '-1:postag': 'XSA', '-1:word.len': 2, '+1:postag': 'XSA', '+1:word.len': 2}
{'bias': 1.0, 'postag': 'ECE', 'dict_pos': False, 'dict_neg': False, 'word.len'

{'bias': 1.0, 'postag': 'NNG', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '-1:postag': 'NNG', '-1:word.len': 2, '+1:postag': 'NNG', '+1:word.len': 2}
{'bias': 1.0, 'postag': 'JKM', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'JKM', '-1:word.len': 1, '+1:postag': 'JKM', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'NNG', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '-1:postag': 'NNG', '-1:word.len': 2, '+1:postag': 'NNG', '+1:word.len': 2}
{'bias': 1.0, 'postag': 'JKS', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'JKS', '-1:word.len': 1, '+1:postag': 'JKS', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'VA', 'dict_pos': False, 'dict_neg': False, 'word.len': 1, '-1:postag': 'VA', '-1:word.len': 1, '+1:postag': 'VA', '+1:word.len': 1}
{'bias': 1.0, 'postag': 'ECE', 'dict_pos': False, 'dict_neg': False, 'word.len': 2, '-1:postag': 'ECE', '-1:word.len': 2, '+1:postag': 'ECE', '+1:word.len': 2}
{'bias': 1.0, 'postag': 'VV', 'dict_pos': F

In [78]:
y

[['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0'],
 ['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'],
 ['100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100'],
 ['100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100',
  '100'],
 ['100',
  '100',
  '100',
  '100'

In [79]:
print(len(X))
print(len(y))
print(len(sentences))

300
300
300


In [80]:
#10번째 sentence의 정보 
print(len(X[10])) 
print(len(y[10]))
print(sentences[10])
print(y[10])
print(X[10])

36
36
[('기간', 'NNG', '100'), ('제', 'XSN', '100'), ('로', 'JKM', '100'), ('일하', 'VV', '100'), ('며', 'ECE', '100'), ('열심히', 'MAG', '100'), ('하', 'VV', '100'), ('었', 'EPT', '100'), ('으나', 'ECE', '100'), ('남자', 'NNG', '100'), ('가', 'JKS', '100'), ('일하', 'VV', '100'), ('기', 'ETN', '100'), ('는', 'JKS', '100'), ('별로', 'MAG', '100'), ('일', 'VV', '100'), ('ㄴ', 'ETD', '100'), ('듯하', 'VXA', '100'), ('ㅂ니다', 'EFN', '100'), ('.', 'SF', '100'), ('여자', 'NNG', '100'), ('가', 'JKS', '100'), ('너무', 'MAG', '100'), ('많', 'VA', '100'), ('아', 'ECD', '100'), ('힘들', 'VA', '100'), ('ㄴ', 'ETD', '100'), ('일', 'NNG', '100'), ('은', 'JX', '100'), ('남자', 'NNG', '100'), ('기간', 'NNG', '100'), ('제가', 'NNG', '100'), ('많이', 'MAG', '100'), ('하', 'VV', '100'), ('는', 'ETD', '100'), ('듯', 'NNB', '100')]
['100', '100', '100', '100', '100', '100', '100', '100', '100', '100', '100', '100', '100', '100', '100', '100', '100', '100', '100', '100', '100', '100', '100', '100', '100', '100', '100', '100', '100', '100', '100', '100', '10

In [81]:
# 문장 하나 확인
# 각 단어의 features가 dictionary로 표현되고 이를 요소로 하는 리스트
X[0]
# 문장은 단어로 이루어져 있고, 각 단어의 features 확인 가능
# 첫 번째 단어는 앞 단어에 대한 정보는 없고 BOS = true

[{'bias': 1.0,
  'postag': 'NNG',
  'dict_pos': False,
  'dict_neg': False,
  'word.len': 2,
  '+1:postag': 'NNG',
  '+1:word.len': 2},
 {'bias': 1.0,
  'postag': 'VV',
  'dict_pos': False,
  'dict_neg': False,
  'word.len': 2,
  '-1:postag': 'VV',
  '-1:word.len': 2,
  '+1:postag': 'VV',
  '+1:word.len': 2},
 {'bias': 1.0,
  'postag': 'ECE',
  'dict_pos': False,
  'dict_neg': False,
  'word.len': 1,
  '-1:postag': 'ECE',
  '-1:word.len': 1,
  '+1:postag': 'ECE',
  '+1:word.len': 1},
 {'bias': 1.0,
  'postag': 'NNG',
  'dict_pos': False,
  'dict_neg': False,
  'word.len': 3,
  '-1:postag': 'NNG',
  '-1:word.len': 3,
  '+1:postag': 'NNG',
  '+1:word.len': 3},
 {'bias': 1.0,
  'postag': 'JKM',
  'dict_pos': False,
  'dict_neg': False,
  'word.len': 2,
  '-1:postag': 'JKM',
  '-1:word.len': 2,
  '+1:postag': 'JKM',
  '+1:word.len': 2},
 {'bias': 1.0,
  'postag': 'VV',
  'dict_pos': False,
  'dict_neg': False,
  'word.len': 1,
  '-1:postag': 'VV',
  '-1:word.len': 1,
  '+1:postag': 'VV',
 

In [82]:
print(len(X), len(X)*0.7) #train-test split (70%, 30%)
print(type(X))

300 210.0
<class 'list'>


In [83]:
crf = CRF(algorithm='lbfgs',
          c1=0.1, #c1의 가중치를 조절하여 feature의 영향력 조절
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [84]:

# Training
crf.fit(X[:300], y[:300])

c:\users\sj\appdata\local\programs\python\python39\lib\site-packages\sklearn\base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CRF(algorithm='lbfgs', all_possible_transitions=False, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [85]:
# cross validation
pred = cross_val_predict(estimator=crf, X=X, y=y, cv=5) 

In [86]:
# prediction
y_test_pred = crf.predict(X[300:])

In [87]:
# report
report = flat_classification_report(y_pred=pred, y_true=y)
print(report)

# data + deep learning 적용 시 늘어날 가능성은 있음. 

              precision    recall  f1-score   support

           0       0.50      0.51      0.50      3811
         100       0.45      0.44      0.45      3520

    accuracy                           0.48      7331
   macro avg       0.47      0.47      0.47      7331
weighted avg       0.48      0.48      0.48      7331



c:\users\sj\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass labels=None as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [88]:
# 태그 간의 transition(전이) probabilities, 태그 별 예측에 중요한 features
# eli5: weight visualization
eli5.show_weights(crf, top=30) # -> 중요한 것만 확인

In [93]:
# 시각화 하려고 했지만 실패한 흔적

explainer = LimeTextExplainer(class_names='CRF_NER')

In [92]:
exp1 = explainer.explain_instance(text1[0], pred, num_features=5)

TypeError: 'numpy.ndarray' object is not callable